In [ ]:
%reload_ext autoreload
%autoreload 2

from glob import glob
import random
import pandas as pd
from pathlib import Path
import sys
sys.path.append('src')
import src.datagen_aug as dg
import cv2
from PIL import Image
jpgs =[]
#jpgs  += glob('./parkunbo/preprocessed_pwb_3/**/*.jpg', recursive=True)
#jpgs  += glob('./hunet/preprocessed_hunet/Est_Soft_01_1.mov.03-16.ss-01-28.to-02-02_000/*.jpg', recursive=True)
jpgs  += glob('./hunet/preprocess/손동작_사복_측면_000/*.jpg', recursive=True)
print(len(jpgs))
#jpgs  += glob('./hunet/preprocessed_hunet/Est_Soft_01_1.mov.00-16.ss-00-00.to-00-32_000/*.jpg', recursive=True)

#front_jpgs  = glob('./parkunbo/preprocessed_pwb_2/12_#2_thursday_02.00_30_000/*.jpg')
len(jpgs)

In [ ]:
front = jpgs
side = jpgs
len(front), len(side)

In [ ]:
from src.mask_history import calc_poly

def mask_img(jpg, mask_ver, randomness):
    img_gt = cv2.imread(jpg)
    sidx = int(Path(jpg).stem.split('_')[0])
    preds = dg.LipGanDS.read_pickle(None, Path(jpg).parent)
    pts = calc_poly[mask_ver](preds[sidx], img_gt.shape[0], randomness=randomness)
    masked = dg.masking(img_gt.copy(), pts)
    return img_gt[:,:,[2,1,0]], masked[:,:,[2,1,0]]
    
def show_mask(jpg, mask_ver, randomness=True):
    img_gt, masked = mask_img(jpg, mask_ver=mask_ver, randomness=randomness)
    img_gt = Image.fromarray(img_gt)
    masked = Image.fromarray(masked)
    img_gt.thumbnail((300, 300))
    masked.thumbnail((300, 300))
    display(img_gt)
    display(masked)
    #print(img_gt.size)
    #print(jpg)
    
mask_ver = 9
#mask_ver = 'pwb_front_v39_1'
mask_ver = 'pwb_side_v39_12'



show_mask(random.choice(side), mask_ver=mask_ver, randomness=True )

#mask_ver = 'pwb_front_v39_1'
#show_mask(random.choice(front), mask_ver=mask_ver, randomness=True )
#show_mask(random.choice(front), randomness=False )

In [ ]:
from tqdm.auto import tqdm

videos  = glob('./hunet/preprocess/*측면*')
v = random.choice(videos)
print(v)
video_jpgs = sorted(glob(f'{v}/*.jpg', recursive=True))
print(video_jpgs[0])

video_jpgs = sorted(video_jpgs)
mask_ver = 'pwb_side_v39_12'
masked = [mask_img(e, mask_ver=mask_ver, randomness=False)[1] for e in tqdm(video_jpgs)]

In [ ]:
import moviepy.editor as mpe
import IPython.display as ipd
vc = mpe.ImageSequenceClip(masked, fps=29.97)
vc.write_videofile('temp.mp4', logger=None)

In [ ]:
ipd.HTML("""<video src="temp.mp4" controls/>""")

In [ ]:
#!pip install ipywidgets